In [1]:
# do basic imports and unpack McMurdo data

#from pmagpy import ipmag
#reload(ipmag)
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
reload(data_model3)
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

import pmagpy.controlled_vocabularies3 as cv



/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
got full_df
got parsed_df


In [2]:
dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
con = Contribution(dir_path)

loc_dm = con.tables['locations'].data_model.dm['locations']
loc_df = con.tables['locations'].df
site_dm = con.tables['sites'].data_model.dm['sites']
site_df = con.tables['sites'].df
samp_df = con.tables['samples'].df
samp_dm = con.tables['samples'].data_model.dm['samples']
spec_df = con.tables['specimens'].df
spec_dm = con.tables['specimens'].data_model.dm['specimens']
age_df = con.tables['ages'].df
age_dm = con.tables['ages'].data_model.dm['ages']
meas_df = con.tables['measurements'].df
meas_dm = con.tables['measurements'].data_model.dm['measurements']
cont_df = con.tables['contribution'].df
cont_dm = con.tables['contribution'].data_model.dm['contribution']
crit_df = con.tables['criteria'].df
crit_dm = con.tables['criteria'].data_model.dm['criteria']


current_con = con

# mess up some validations for locations
loc_df.loc['Tel Hazor', 'lat_s'] = 400.
loc_df['dir_inc'] = 5
loc_df.loc['Tel Hazor', 'lat_n'] = 'hello'
loc_df.loc[:, 'lithologies'] = ["Agate:Basalt", "Basalt:random"]
#current_con.tables.pop('sites')

# mess up some validations for sites
site_df.pop('age')
site_df['dir_tilt_correction'] = 1
site_df['dir_tilt_correction'] = 'not a number'
site_df.iloc[0, list(site_df.columns).index('lithologies')] = "Angrite:Basalt"
site_df.iloc[1, list(site_df.columns).index('lithologies')] = "angrite : basalt"

# mess up some validations for ages
age_df.ix[1]['age'] = 'a string'
age_df.ix[1]['site'] = 'fake site'
age_df.ix[1]['age_low'] = 1000000000000.
age_df.pop('citations')

# mess up some validations for samples
samp_df.pop('citations')
samp_df.iloc[0].lon = 600.
samp_df.iloc[0].age = "another string"
samp_df.iloc[0].lat = "stringy"
samp_df.iloc[1].lat = 'hello'
samp_df.iloc[2].specimens = "hz05a2:fake"
samp_df.iloc[3].specimens = "fake : hz05a1"
samp_df.iloc[5].specimens = 'fake_specimen'
samp_df.iloc[7].site = 'fake_site'
samp_df.iloc[0].cooling_rate = 'a string'

# mess up some validations for measurements
meas_df.loc['mgh05a01:LP-PI-TRM1', 'magn_moment'] = 2
meas_df.loc['mgh05a01:LP-PI-TRM1', 'specimen'] = "fake_specimen"
meas_df.pop('experiment')

#current_df.head()
#current_df.head()
age_df.head()

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt


,age,age_high,age_low,age_unit,description,location,method_codes,site
0,-740,-732,-800,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz05
1,a string,-732,1e+12,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,fake site
2,-850,-800,-900,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz07
3,-950,-900,-1000,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz09
4,-950,-900,-1000,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz10


In [3]:
import pmagpy.controlled_vocabularies3 as cv
reload(cv)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()

-I- Importing controlled vocabularies from https://earthref.org


In [4]:
## validation functions


# need to add requiredOneInGroup




def requiredUnless(col_name, arg, dm, df):
    """
    Arg is a string in the format "str1, str2, ..."
    Each string will be a column name.
    Col_name is required in df unless each column from arg is present.
    """
    arg_list = arg.split(",")
    arg_list = [arg.strip('"') for arg in arg_list]
    msg = ""
    for a in arg_list:
        # ignore validations that reference a different table
        if "." in a:
            continue
        if a not in df.columns:
            msg += "{} column is required unless {} is present.  ".format(col_name, a)
    if msg:
        return msg
    else:
        return None
    return None


def requiredUnlessTable(col_name, arg, dm, df):
    """
    Col_name must be present in df unless
    arg (table_name) is present in contribution
    """
    table_name = arg
    if col_name in df.columns:
        return None
    elif table_name in current_con.tables:
        return None
    else:
        return "{} column is required unless table {} is present".format(col_name, table_name)

    
def requiredIfGroup(col_name, arg, dm, df):
    """
    Col_name is required if other columns of 
    the group arg are present.
    """
    group_name = arg
    groups = set()
    columns = df.columns
    for col in columns:
        if col not in dm.index:
            continue
        group = dm.loc[col]['group']
        groups.add(group)
    if group_name in groups:
        if col_name in columns:
            return None
        else:
            return "{} column is required if column group {} is used".format(col_name, group_name)
    return None


def required(col_name, arg, dm, df):
    """
    Col_name is required in df.columns.
    Return error message if not.
    """
    if col_name in df.columns:
        return None
    else:
        return '"{}" column is required'.format(col_name) 

def isIn(row, col_name, arg, dm, df):
    """
    row[col_name] must contain a value from another column.
    If not, return error message.
    """
    #grade = df.apply(func, args=(validation_name, arg, dm), axis=1)
    x = 0
    cell_value = row[col_name]
    if not cell_value:
        return None
    # if it's in another table
    cell_values = [v.strip(" ") for v in cell_value.split(":")]
    if "." in arg:
        table_name, table_col_name = arg.split(".")
        if table_name not in current_con.tables:
            return "Must contain a value from {} table. Missing {} table.".format(table_name, table_name)
        if table_col_name not in current_con.tables[table_name].df.columns:
            return '{} table is missing "{}" column, which is required for validating "{}" column'.format(table_name, table_col_name, col_name)
        possible_values = current_con.tables[table_name].df[table_col_name].unique()
        for value in cell_values:
            if value not in possible_values:
                return 'This value: "{}" is not found in: {}'.format(value, arg)
                break
    # if it's in the present table:
    else:
        possible_values = df[arg].unique()
        for value in cell_values:
            if value not in possible_values:
                return 'This value: "{}" is not found in: {} column'.format(value, arg)
                break
    return None
    
def checkMax(row, col_name, arg, *args):
    """
    row[col_name] must be less than or equal to arg.
    else, return error message.
    """
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg_val = float(arg)
    except ValueError:
        arg_val = row[arg]
    #arg = float(arg)
    try:
        if float(cell_value) <= float(arg_val):
            return None
        else:
            return "{} ({}) must be <= {} ({})".format(str(cell_value), col_name, str(arg_val), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def checkMin(row, col_name, arg, *args):
    """
    row[col_name] must be greater than or equal to arg.
    else, return error message.
    """
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg_val = float(arg)
    except ValueError:
        arg_val = row[arg]
    try:
        if float(cell_value) >= float(arg_val):
            return None
        else:
            return "{} ({}) must be >= {} ({})".format(str(cell_value), col_name, arg_val, str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def cv(row, col_name, arg, current_data_model, *args):
    """
    row[col_name] must contain only values from the appropriate controlled vocabulary
    """
    cell_value = row[col_name]
    if not cell_value:
        return None
    cell_values = cell_value.split(":")
    cell_values = [c.strip() for c in cell_values]
    for value in cell_values:
        if value.lower() in [v.lower() for v in vocabulary[col_name]]:
            continue
        else:
            return '"{}" is not in controlled vocabulary for {}'.format(value, arg)
    return None
        

# validate presence
presence_operations = {"required": required, "requiredUnless": requiredUnless,
                       "requiredIfGroup": requiredIfGroup, 
                       'requiredUnlessTable': requiredUnlessTable}
# validate values
value_operations = {"max": checkMax, "min": checkMin, "cv": cv, "in": isIn}

def split_func(string):
    """
    Take a string like 'requiredIf("arg_name")'
    return the function name and the argument:
    (requiredIf, arg_name)
    """
    ind = string.index("(")
    return string[:ind], string[ind+1:-1].strip('"')


def test_type(value, value_type):
    if not value:
        return None
    if value_type == "String":
        if str(value) == value:
            return None
        else:
            return "should be string"
    elif value_type == "Number":
        try:
            float(value)
            return None
        except ValueError:
            return '"{}" should be a number'.format(str(value))
    elif value_type == "Integer":
        if isinstance(value, str):
            if str(int(value)) == value:
                return None
            else:
                return '"{}" should be an integer'.format(str(value))
        else:
            if int(value) == value:
                return None
            else:
                return '"{}" should be an integer'.format(str(value))
    else:
        return None
    #String, Number, Integer, List, Matrix, Dictionary, Text
    


def validate_df(df, dm):
    # check column validity
    cols = df.columns
    invalid_cols = [col for col in cols if col not in dm.index]
    for validation_name, validation in dm.iterrows():
        value_type = validation['type']
        if validation_name in df.columns:
            output = df[validation_name].apply(test_type, args=(value_type,))
            df["type_pass" + "_" + validation_name + "_" + value_type] = output

        val_list = validation['validations']
        if not val_list or isinstance(val_list, float):
            continue
        for num, val in enumerate(val_list):
            func_name, arg = split_func(val)
            if arg == "magic_table_column":
                continue
            # first validate for presence
            if func_name in presence_operations:
                func = presence_operations[func_name]
                #grade = func(validation_name, df, arg, dm)
                grade = func(validation_name, arg, dm, df)
                pass_col_name = "presence_pass_" + validation_name + "_" + func.__name__
                df[pass_col_name] = grade
    
            # then validate for correct values
            elif func_name in value_operations:
                func = value_operations[func_name]
                if validation_name in df.columns:
                    grade = df.apply(func, args=(validation_name, arg, dm, df), axis=1)
                    col_name = "value_pass_" + validation_name + "_" + func.__name__
                    if col_name in df.columns:
                        num_range = range(1, 10)
                        for num in num_range:
                            if (col_name + str(num)) in df.columns:
                                continue
                            else:
                                col_name = col_name + str(num)
                                break
                    df[col_name] = grade.astype(object)
    return df

  
    


    

# check that values pass validation
# validation checks to add:
# sv (suggested vocab)
# requiredOneInGroup
# requiredUnlessSynthetic


# re-do upload_magic to use contribution-level (??)

# first, do validations on each table in the contribution
# this will include removing unneeded data (RmKeys from old upload_magic)
# this will also include checking everything against the data model (strings are strings, etc.)g


# next, splat out each table into a file and wrap it up.  give it a sensible name.  

In [5]:
#current_df = current_con.tables['sites'].df  
#current_dm = current_con.tables['sites'].data_model.dm['sites']

#current_df = validate_df(current_df, current_dm)

In [6]:
def get_validation_col_names(df):
    value_cols = df.columns.str.match("^value_pass_")
    present_cols = df.columns.str.match("^presence_pass")
    type_cols = df.columns.str.match("^type_pass_")

    value_col_names = df.columns[value_cols]
    present_col_names = df.columns[present_cols]
    type_col_names = df.columns[type_cols]

    validation_cols = np.where(value_cols, value_cols, present_cols)
    validation_cols = np.where(validation_cols, validation_cols, type_cols)
    validation_col_names = df.columns[validation_cols]
    return value_col_names, present_col_names, type_col_names, validation_col_names

#value_col_names, present_col_names, type_col_names, validation_col_names = get_validation_col_names(current_df)
#present_col_names

In [7]:
# incorrect data type problems
#current_df[type_col_names].dropna(how='all', axis=1).head()

In [8]:
# missing column problems
#current_df[present_col_names].dropna(how='all', axis=1).head()

In [9]:
# value problems:
#current_df[value_col_names].dropna(how='all', axis=1).head()

In [10]:
def extract_col_name(string):
    prefixes = ["presence_pass_", "value_pass_", "type_pass_"]
    end = string.rfind("_")
    for prefix in prefixes:
        if string.startswith(prefix):
            return prefix[:-6], string[len(prefix):end]
    return string, string


def check_row(row):
    ind = row[row.notnull()].index
    values = row[row.notnull()].values
    # to transformation with extract_col_name here???
    return dict(zip(ind, values))

#def check_row(row):
#    return True


def print_failures(failing_items, verbose=False, outfile=None):
    if outfile:
        ofile = open(outfile, "w")
        ofile.write("\t".join(["name", "row_number", "problem_type", "problem_col", "error_message"]))
        ofile.write("\n")
    else:
        ofile = None
    for ind, row in failing_items.iterrows():
        issues = row["issues"]
        string = "{:10}  |  row number: {}".format(ind, str(row["num"]))
        first_string = "\t".join([str(ind), str(row["num"])])
        if verbose:
            print first_string
        #if outfile:
        #    ofile.write("{}\n".format(string))
        for key, issue in issues.items():
            issue_type, issue_col = extract_col_name(key)
            string = "{:10}  |  {:10}  |  {}".format(issue_type, issue_col, issue)
            string = "\t".join([issue_type, issue_col, issue])
            if verbose:
                print string
            if outfile:
                ofile.write(first_string + "\t" + string + "\n")
    if outfile:
        ofile.close()

def get_all_failures(df, value_cols, type_cols, verbose=False, outfile=None):
    df["num"] = range(len(df))
    # get column names for value & type validations
    names = value_cols.union(type_cols)
    # drop all non validation columns
    value_problems = df[names.union(["num"])]
    failing_items = value_problems.dropna(how="all", subset=names)
    if not len(failing_items):
        if verbose:
            print "No problems"
        return []
    failing_items = failing_items.dropna(how="all", axis=1)
    # get names of the failing items
    bad_items = list(failing_items.index)
    # get index numbers of the failing items
    bad_indices = list(failing_items["num"])
    zip(bad_indices, bad_items)
    #failing_items.drop("num", axis=1, inplace=True)#.apply(check_row, axis=1).values
    failing_items['issues'] = failing_items.drop("num", axis=1).apply(check_row, axis=1).values
    # maybe do a transformation in here so that you get "lon" instead of "value_pass_lon_checkMax"
    print_failures(failing_items, verbose, outfile)
    return failing_items



In [11]:

def get_bad_rows_and_cols(df, validation_names, type_col_names, value_col_names, verbose=False):
    df["num"] = range(len(df))
    problems = df[validation_names.union(["num"])]
    all_problems = problems.dropna(how='all', axis=0, subset=validation_names)
    value_problems = problems.dropna(how='all', axis=0, subset=type_col_names.union(value_col_names))
    all_problems = all_problems.dropna(how='all', axis=1)
    value_problems = value_problems.dropna(how='all', axis=1)
    if not len(problems):
        return None, None, None
    #
    bad_cols = all_problems.columns
    prefixes = ["value_pass_", "type_pass_"]
    missing_prefix = "presence_pass_"
    problem_cols = []
    missing_cols = []
    long_missing_cols = []
    problem_rows = []
    for col in bad_cols:
        pre, stripped_col = extract_col_name(col)
        for prefix in prefixes:
            if col.startswith(prefix):
                problem_cols.append(stripped_col)
                continue
        if col.startswith(missing_prefix):
            missing_cols.append(stripped_col)
            long_missing_cols.append(col)
            
    if len(value_problems):
        bad_rows = zip(list(value_problems["num"]), list(value_problems.index))
    else:
        bad_rows = []
    if verbose:
        if bad_rows:
            if len(bad_rows) > 20:
                print "-W- these rows have problems:", bad_rows[:20], " ...",
                print "(for full error output see error file)"
            else:
                print "-W- these rows have problems:", bad_rows
        if problem_cols:
            print "-W- these columns contain bad values:", ", ".join(problem_cols)
        if missing_cols:
            print "-W- these required columns are missing:", ", ".join(missing_cols)
    return bad_rows, problem_cols, missing_cols
    
#a, b, c = get_bad_rows_and_cols(current_df, validation_col_names)
#if a:
#    print "bad rows:", a[:10]
#    print "problems:", b[:10]
#    print "missing:", c[:10]

In [12]:
## run through and validate entire contribution

#dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
#con = Contribution(dir_path)
the_con = con


#for dtype in the_con.tables.keys()[2:4]:
# doesn't work as validate_table function
def validate_table(the_con, dtype, verbose=False):
    """
    Return name of bad table, or False if no errors found
    """
    print "-I- Validating {}".format(dtype)
    # grab dataframe
    current_df = the_con.tables[dtype].df
    # grab data model
    current_dm = the_con.tables[dtype].data_model.dm[dtype]
    # run all validations (will add columns to current_df)
    current_df = validate_df(current_df, current_dm)
    # get names of the added columns
    value_col_names, present_col_names, type_col_names, validation_col_names = get_validation_col_names(current_df)
    # print out failure messages
    ofile = os.path.join(os.getcwd(), "{}_errors.txt".format(dtype))
    failing_items = get_all_failures(current_df, value_col_names, type_col_names, verbose, outfile=ofile)
    #x = set(value_col_names).union(type_col_names)
    bad_rows, bad_cols, missing_cols = get_bad_rows_and_cols(current_df, validation_col_names, 
                                                             value_col_names, type_col_names, verbose=True)
    # delete all validation rows
    current_df.drop(validation_col_names, axis=1, inplace=True)
    if len(failing_items):
        print "-I- Complete list of row errors can be found in {}".format(ofile)
        return dtype
    else:
        print "-I- No row errors found!"
        return False
    

    
    
def validate_contribution(the_con):
    passing = True
    for dtype in the_con.tables.keys():#, 'criteria']:
        print "validating {}".format(dtype)
        fail = validate_table(the_con, dtype)
        if fail:
            passing = False
        print '--'


#validate_contribution(current_con)

# work: sites
# doesn't work: specimens, samples, locations
validate_table(the_con, 'locations', verbose=True)

-I- Validating locations
Tel Hazor	0
value	lat_s	400.0 (lat_s) must be <= 90.0 (90)
type	lat_n	"hello" should be a number
Tel Megiddo	1
value	lithologies	"random" is not in controlled vocabulary for lithology
-W- these rows have problems: [(0, 'Tel Hazor'), (1, 'Tel Megiddo')]
-W- these columns contain bad values: lat_n, lat_s, lithologies
-W- these required columns are missing: age_high, age_low, age, age_unit, dir_dec, dir_tilt_correction, geologic_classes
-I- Complete list of row errors can be found in /Users/nebula/Python/PmagPy/locations_errors.txt


'locations'

In [13]:
# adjust all degree columns to 360

def get_degree_cols(df):
    vals = ['lon_w', 'lon_e', 'lat_lon_precision', 'pole_lon', 'paleolon', 'paleolon_sigma', 
            'lon', 'lon_sigma', 'vgp_lon', 'paleo_lon', 'paleo_lon_sigma', 
            'azimuth', 'azimuth_dec_correction', 'dir_dec', 
            'geographic_precision', 'bed_dip_direction']
    relevant_cols = list(set(vals).intersection(df.columns))
    return relevant_cols


def adjust_to_360(arg):
    if not arg:
        return None
    else:
        try:
            return float(arg) % 360
        except ValueError:
            return arg


#samp_df.loc['hz05a', 'lon'] = 370
#relevant_cols = get_degree_cols(samp_df)
#for col in relevant_cols:
#    samp_df[col] = samp_df[col].apply(adjust_to_360)
#samp_df[relevant_cols].head()

In [14]:
import time
from pmagpy import pmag
reload(pmag)
from pmagpy import data_model3 as data_model
reload(nb)


def upload_magic(concat=0, dir_path='.', dmodel=None):
    """                                                                                                                              
    Finds all magic files in a given directory, and compiles them into an upload.txt file which can be uploaded into the MagIC datab\
ase.                                                                                                                                 
    returns a tuple of either: (False, error_message, errors) if there was a problem creating/validating the upload file             
    or: (filename, '', None) if the upload was fully successful                                                                      
    """
    SpecDone=[]
    locations = []
    concat = int(concat)
    dtypes = ["locations", "samples", "specimens", "sites", "ages", "measurements",
                  "criteria", "contribution", "images"]
    file_names = [os.path.join(dir_path, dtype + ".txt") for dtype in dtypes]
    con = Contribution(dir_path)
    # begin the upload process                                                                                                       
    up = os.path.join(dir_path, "upload.txt")
    if os.path.exists(up):
        os.remove(up)
    RmKeys = ['citation_label', 'compilation', 'calculation_type', 'average_n_lines', 'average_n_planes',
              'specimen_grade', 'site_vgp_lat', 'site_vgp_lon', 'direction_type', 'specimen_Z',
              'magic_instrument_codes', 'cooling_rate_corr', 'cooling_rate_mcd', 'anisotropy_atrm_alt',
              'anisotropy_apar_perc', 'anisotropy_F', 'anisotropy_F_crit', 'specimen_scat',
              'specimen_gmax','specimen_frac', 'site_vadm', 'site_lon', 'site_vdm', 'site_lat',
              'measurement_chi', 'specimen_k_prime','specimen_k_prime_sse','external_database_names',
              'external_database_ids', 'Further Notes', 'Typology', 'Notes (Year/Area/Locus/Level)',
              'Site', 'Object Number']
    print "-I- Removing: ", RmKeys
    CheckDec = ['_dec', '_lon', '_azimuth', 'dip_direction']
    CheckSign = ['specimen_b_beta']
    last = file_names[-1]
    methods, first_file = [], 1
    failing = []
    if not dmodel:
        dmodel = data_model.DataModel()
    for file_type in dtypes:
        print "-"
    # read in the data
        #Data, file_type = pmag.magic_read(File)
        if file_type not in con.tables.keys():
            print "-I- No {} file found, continuing".format(file_type)
            continue
        container = con.tables[file_type]
        df = container.df
        if len(df):
            print "-I- {} file successfully read in".format(file_type)
            # make some adjustments to clean up data
            # drop non MagIC keys
            DropKeys = set(RmKeys).intersection(df.columns)
            df.drop(DropKeys, axis=1, inplace=True)
            # make sure int_b_beta is positive
            if 'int_b_beta' in df.columns:
                df['int_b_beta'] = df['int_b_beta'].astype(float).apply(abs)
            # make all declinations/azimuths/longitudes in range 0=>360.
            relevant_cols = get_degree_cols(df)
            for col in relevant_cols:
                df[col] = df[col].apply(adjust_to_360)
            # get list of location names
            if file_type == 'locations':
                locations = sorted(df['location'].unique())              
            ## LJ: leave this for validations??
            # use only highest priority orientation -- not sure how this works
            elif file_type == 'samples':
                #orient,az_type=pmag.get_orient(Data,rec['sample'])
                pass
            # include only specimen records with samples
            elif file_type == 'specimens':
                df = df[df['sample'].notnull()]
                if 'samples' in con.tables:
                    samp_df = con.tables['samples'].df
                    df = df[df['sample'].isin(samp_df.index.unique())]
            # include only measurements with specmiens
            elif file_type == 'measurements':
                df = df[df['specimen'].notnull()]
                if 'specimens' in con.tables:
                    spec_df = con.tables['specimens'].df
                    df = df[df['specimen'].isin(spec_df.index.unique())]
            ## LJ: should we still do this??
            #if file_type == 'samples': # check to only upload top priority orientation record!                                      
            #    NewSamps, Done = [], []
            #    for rec in Data:
            #        if rec['sample'] not in Done:
            #            #orient,az_type=pmag.get_orient(Data,rec['sample'])
            #            #NewSamps.append(orient)
            #            Done.append(rec['sample'])
            #    Data=NewSamps
            #    print 'only highest priority orientation record from samples.txt read in '
            #if file_type == 'specimens': #  only specimens that have sample names                                                   
            #    NewData,SpecDone=[],[]
            #    for rec in Data:
            #        if rec['sample'] in Done:
            #            NewData.append(rec)
            #            SpecDone.append(rec['specimen'])
            #        else:
            #            print 'no valid sample record found for: '
            #            print rec
            #    Data = NewData
            #    #print 'only measurements that have specimen/sample info'  
                
            ## LJ: leave this for validations??
            #if file_type == 'measurements': #  only measurements that have specimen names                                        
            #    no_specs = []
            #    NewData = []
            #    for rec in Data:
            #        if rec['specimen'] in SpecDone:
            #            NewData.append(rec)
            #        else:
            #            print 'no valid specimen record found for: '
            #            print rec    
            #            no_specs.append(rec)
            #    #print set([record['er_specimen_name'] for record in no_specs])                                                      
            #    Data = NewData
                
    # run validations
            res = validate_table(con, file_type)#, verbose=True)
            if res:
                failing.append(res)
    # write out the data
            if len(df):
                container.write_magic_file(up, append=True)
    # write out the file separator                                                                                                   
            f = open(up, 'a')
            f.write('>>>>>>>>>>\n')
            f.close()
            print "-I-", file_type, 'written to ',up
        else:
            print 'File:', File
            print file_type, 'is bad or non-existent - skipping '
    ## add to existing file
    if concat == 1:
        f = open(up, 'a')
        f.write('>>>>>>>>>>\n')
        f.close()
     
    if not os.path.isfile(up):
        print "no data found, upload file not created"
        return False, "no data found, upload file not created", None

    #rename upload.txt according to location + timestamp                                                                             
    format_string = "%d.%b.%Y"
    if locations:
        locs = set(locations)
        locs = sorted(locs)[:3]
        #location = locations[0].replace(' ', '_')
        locs = [loc.replace(' ', '-') for loc in locs]
        location = "_".join(locs)
        new_up = location + '_' + time.strftime(format_string) + '.txt'
    else:
        new_up = 'unknown_location_' + time.strftime(format_string) + '.txt'

    new_up = os.path.join(dir_path, new_up)
    if os.path.isfile(new_up):
        fname, extension = os.path.splitext(new_up)
        for i in range(1, 100):
            if os.path.isfile(fname + "_" + str(i) + extension):
                continue
            else:
                new_up = fname + "_" + str(i) + extension
                break
    if not up:
        print "-W- Could not create an upload file"
        return
    os.rename(up, new_up)
    print "Finished preparing upload file: {} ".format(new_up)
    ## LJ: add back in validation stuff
    if failing:
        print "-W- validation of upload file has failed."
        print "These tables have errors: {}".format(", ".join(failing))
        print "Please fix above errors and try again."
        print "You may run into problems if you try to upload this file to the MagIC database."
    #    return False, "file validation has failed.  You may run into problems if you try to upload this file.", errors
    else:
        print "-I- Your file has passed validation.  Yay!"
    return new_up, '', None


dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
#dir_path = os.path.join(os.getcwd(), '3_0', 'Osler')
#dir_path = os.path.join(os.getcwd(), '3_0', 'McMurdo')

upload_magic(dir_path=dir_path)

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt
-I- Removing:  ['citation_label', 'compilation', 'calculation_type', 'average_n_lines', 'average_n_planes', 'specimen_grade', 'site_vgp_lat', 'site_vgp_lon', 'direction_type', 'specimen_Z', 'magic_instrument_codes', 'cooling_rate_corr', 'cooling_rate_mcd', 'anisotropy_atrm_alt', 'anisotropy_apar_perc', 'anisotropy_F', 'anisotropy_F_crit', 'specimen_scat', 'specimen_gmax', 'specimen_frac', 'site_vadm', 'site_lon', 'site_vdm', 'site_lat', 'measurement_chi', 'specimen_k_prime', 'specimen_k_prime_sse', 'external_database_names', 'external_database_ids', 'Further Notes', 'Typology', 'Notes (Year/Area/Locus/Level)', 'Site', 'Object Number']
-
-I- locations file successfully read in
-I- Validating locations
-W- these required columns are missing: age_high, age_low, age, age_unit, geologic_classes, lithologies
-I- No row errors found!
-I- appending locations data to /Users/nebula/Python/PmagPy/3_0/Megiddo/upload.txt
-I- locat

('/Users/nebula/Python/PmagPy/3_0/Megiddo/Tel-Hazor_Tel-Megiddo_28.Jul.2016_5.txt',
 '',
 None)

In [15]:
'hello.txt'[:-4]

'hello'

## Filling in an existing dataframe

In [16]:
# keep all of df1, add in any extra from df2
df1 = pd.DataFrame(np.random.randint(1, 10, (3, 5)), columns=['one', 'two', 'three', 'four', 'five'])
df1.iloc[0, 1] = np.nan
df1.iloc[2, 2] = np.nan
df2 = pd.DataFrame(np.random.randint(1, 10, (3, 5)), columns=['one', 'three', 'five', 'seven', 'nine'])
df1

,one,two,three,four,five
0,7,NaN,1.0,8,6
1,5,1.0,6.0,5,2
2,8,7.0,NaN,6,8


In [17]:
df2

,one,three,five,seven,nine
0,6,4,5,3,4
1,6,3,3,8,2
2,7,1,2,6,5


In [18]:
unique_df2_cols = df2.columns.difference(df1.columns)
unique_df2 = df2[unique_df2_cols]

# this adds in all the unique columns that weren't in df1
concat_df = pd.concat([df1, unique_df2], axis=1)
# fills in null values in df1 with values from df2
concat_df.fillna(df2)

,one,two,three,four,five,nine,seven
0,7,NaN,1.0,8,6,4,3
1,5,1.0,6.0,5,2,2,8
2,8,7.0,1.0,6,8,5,6


In [19]:
! rm *_errors.txt
! rm ./3_0/McMurdo/McMurdo*.txt
#! rm ./3_0/Megiddo/Tel-Hazor*.txt

rm: ./3_0/McMurdo/McMurdo*.txt: No such file or directory
